# Notes

This assignment is devoted to `pandas`. It covers indexing and filtering, and some `groupby` and `join` operations. The assignment roughly corresponds to Week 4 and the beginning of Week 5 of the course.

The main dataset you'll be using is [Titanic](https://www.kaggle.com/c/titanic). Please, note, that you must not rely on any specific location for the dataset, hence, any code like

```python
titanic_train = pd.read_csv("<location>/train.csv")
```

will fail and your notebook won't be validated and graded. Inputs to the functions are described explicitly in each case, and that's the only thing you can rely on.

In [40]:
%pylab inline
plt.style.use("bmh")

Populating the interactive namespace from numpy and matplotlib


In [41]:
plt.rcParams["figure.figsize"] = (6,6)

In [42]:
import numpy as np
import pandas as pd

In [43]:
STUDENT = "Lior Treiman and Hadas Neuman"
ASSIGNMENT = 4
TEST = False

In [44]:
if TEST:
    import solutions
    total_grade = 0
    MAX_POINTS = 16

# Indexing and filtering

### 1. Fixing age (1 point).

There are several known mistakes in the Titanic dataset.

Namely, [Julia Florence Siegel](https://www.encyclopedia-titanica.org/titanic-survivor/julia-florence-cavendish.html) (Mrs. Tyrell William Cavendish) is mistakenly marked as being 76 years old (the age she actually died, but many years after Titanic).

You must **replace the corresponding age value in the dataframe with her actual age at the time** (25) and return the dataset. Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. You must return a copy of the input dataframe, and not perform replacement in the original dataframe. Structure and indexing must be the same as in input.

In [45]:
# Uploading data

if not TEST:
    from google.colab import drive
    drive.mount('/content/drive')
    dir = 'drive/MyDrive/Y-data/Py4DP/L4/'
    train_df = pd.read_csv(dir + 'train.csv', index_col='PassengerId')
    test_df = pd.read_csv(dir + 'test.csv', index_col='PassengerId')

    titanic = pd.concat([train_df, test_df], axis=0)  # Concat by rows

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
def fix_age(df):
    """Fix age for Julia Florence Siegel."""
    # Finding Julia's row
    jname = 'Julia Florence Siegel'
    jindex = df.index[df.Name.str.contains(jname, regex=False)].tolist()
    corrected_df = df.copy()
    corrected_df.loc[jindex, 'Age'] = 25
    return corrected_df

In [47]:
PROBLEM_ID = 1

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

In [48]:
if not TEST:
  cdf = fix_age(titanic)
  print(cdf.loc[988,:])
  print(titanic.loc[988,:])

Survived                                                  NaN
Pclass                                                      1
Name        Cavendish, Mrs. Tyrell William (Julia Florence...
Sex                                                    female
Age                                                      25.0
SibSp                                                       1
Parch                                                       0
Ticket                                                  19877
Fare                                                    78.85
Cabin                                                     C46
Embarked                                                    S
Name: 988, dtype: object
Survived                                                  NaN
Pclass                                                      1
Name        Cavendish, Mrs. Tyrell William (Julia Florence...
Sex                                                    female
Age                                          

### 2. Embarkment port distribution (1 point).

You must find the value counts for embarkment port (`Embarked` column) for the passengers, who travelled in 3-d class, were male and between 20 and 30 years old (both inclusive). No need to treat missing values separately.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. You must return **series**, indexed with values from `Embarked`, according to `.value_counts()` method semantics:

```
S    <number of male passengers in 3-d class, embarked at S, 20<=Age<=30>
C    <...>
Q    <...>
Name: Embarked, dtype: int64
```

In [49]:
def embarked_stats(df):
    """Calculate embarkment port statistics."""
    target_df = df.loc[(df.Pclass == 3) & (df.Sex == 'male') & (df.Age >= 20) & (df.Age <= 30), :]
    embarkes = target_df['Embarked'].value_counts()
    return embarkes

In [50]:
PROBLEM_ID = 2

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, embarked_stats)

In [51]:
if not TEST:
    r = embarked_stats(titanic)
    print(r)

S    132
C     21
Q      7
Name: Embarked, dtype: int64


### 3. Fill missing age values (1 point).

Some age values are missing in the Titanic dataset. You need to calculate average age over all passengers, and fill missing age values in `Age` column.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **new** dataframe with the same structure, but without missing values in `Age` column.

In [52]:
def fix_age(df):
    """Fix missing age values."""
    df_result = df.copy()
    df_result[df_result.Age.isnull()] = df.Age.mean()
    return df_result

In [53]:
PROBLEM_ID = 3

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

In [54]:
if not TEST:
    cdf = fix_age(titanic)
    print(cdf[cdf.Age.isna()])
    print(cdf.Age)


Empty DataFrame
Columns: [Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked]
Index: []
PassengerId
1       22.000000
2       38.000000
3       26.000000
4       35.000000
5       35.000000
          ...    
1305    29.881138
1306    39.000000
1307    38.500000
1308    29.881138
1309    29.881138
Name: Age, Length: 1309, dtype: float64


### 4. Child travelling alone (1 point).

You must find a child (`Age<10`) on-board, who was travelling without siblings or parents and find a name of her nursemaid.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **tuple** of two strings, collected from `Name` column, with one being child's name and second being nursemaid's name. It's known, that there's **only one child** like this.

In [55]:
def get_nursemaid(df):
    # We can find the nursemaid using the ticket number
    child_name = df.loc[(df.Age<10) & (df.SibSp==0) & (df.Parch==0), 'Name'].values[0]
    ticket_num = df.loc[df.Name==child_name, 'Ticket'].values[0]
    nurs_name = df.loc[(df.Ticket==ticket_num) & (df.Name != child_name), 'Name'].values[0]
    return child_name, nurs_name

In [56]:
PROBLEM_ID = 4

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_nursemaid)

In [57]:
if not TEST:
    print(get_nursemaid(titanic))

('Emanuel, Miss. Virginia Ethel', 'Dowdell, Miss. Elizabeth')


### 5. Port with the most children embarked (1 point).

You must find, which port had the largest percentage of children (`Age<10`) embarked, i.e. number of children divided by total number of passengers embarked.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **single string** with port letter.

In [58]:
def get_port(df):
    """Get port with the most children embarked."""
    pass_per_port = df.Embarked.value_counts()
    chls_per_port = df.loc[(df.Age < 10)]['Embarked'].value_counts()
    percent_child = chls_per_port / pass_per_port
    return percent_child.idxmax()

In [59]:
if not TEST:
    print(get_port(titanic))

S


In [60]:
PROBLEM_ID = 5

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_port)

### 6. Passengers per ticket (2 points).

Calculate average and maximum number of passengers per ticket.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **tuple** of two values - average and maximum number of passengers per ticket.

In [61]:
def get_ticket_stats(df):
    """Calculate passenger per ticket statistics."""
    ticket_groups = df.groupby(['Ticket']).size()
    mean_ticket = ticket_groups.mean()
    max_ticket = ticket_groups.max()
    return mean_ticket, max_ticket

In [62]:
PROBLEM_ID = 6

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_ticket_stats)

In [63]:
if not TEST:
    print(get_ticket_stats(titanic))

(1.4090419806243273, 11)


### 7. Fare per passenger (3 points).

For each individual ticket, you must calculate **fare per person for that ticket**, and then calculate averages for each class. Note, that you will need to apply `groupby` and you may consider using `.first()` of resulting `DataFrameGroupBy`. Also, caferully consider, in which order calculations are performed.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be `pd.Series` with three elements, indexed by class:

```
1    <average per person fare in class 1>
2    <...>
3    <...>
Name: Pclass, dtype: float64
```

In [64]:
def get_fare_per_pass(df):
    """Calculate fare per passenger for different classes."""
    # Claculating the fare per passenger
    fare_per_pass = (df.groupby("Ticket")["Fare"].mean() / df.groupby("Ticket").size()).rename("fare_per_pass")
    # Merging back and grouping by 'Pclass'
    fare_per_pass_by_class = df.merge(fare_per_pass, left_on="Ticket", right_index=True, how="left").groupby("Pclass")["fare_per_pass"].mean()
    # Replacing the series name to Pclass for the output format (and removing index name)
    fare_per_pass_by_class.rename('Pclass', inplace=True)
    fare_per_pass_by_class.index.name = None
    return fare_per_pass_by_class

In [65]:
PROBLEM_ID = 7

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_fare_per_pass)

In [66]:
if not TEST:
    print(get_fare_per_pass(titanic))

1    33.910500
2    11.411010
3     7.329146
Name: Pclass, dtype: float64


### 8. Fill missing age values (3 points).

In problem 3 you filled missing age values with global average over all passengers. Now, you need to fill them **according to class and sex**. For example, for a female passenger from 2d class, missing age value must be filled with average age of females in 2d class.

In this problem, you may need joins and `.apply()`, although there are several ways to get the same result.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **new** dataframe with the same structure as input, but without missing values in `Age` column.

In [67]:
def fix_age_groupped(df):
    """Fill missing age values."""
    # Getting the updated age averages
    age_by_sc = df.Age.groupby([df.Pclass, df.Sex]).mean()
    # Copying the dataframe
    new_df = df.copy()
    # Replacing Nan values to the average using the above series
    new_df['Age'] = df.apply(lambda row: age_by_sc[row['Pclass'], row['Sex']] if (pd.isnull(row['Age'])) else row['Age'], axis=1) 
    return new_df

In [68]:
PROBLEM_ID = 8

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age_groupped)

In [69]:
if not TEST:
    temp_df = fix_age_groupped(titanic)

    print(temp_df.info())
    print(temp_df[['Age', 'Sex', 'Pclass']].head(20))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 1 to 1309
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    float64
 1   Pclass    1309 non-null   int64  
 2   Name      1309 non-null   object 
 3   Sex       1309 non-null   object 
 4   Age       1309 non-null   float64
 5   SibSp     1309 non-null   int64  
 6   Parch     1309 non-null   int64  
 7   Ticket    1309 non-null   object 
 8   Fare      1308 non-null   float64
 9   Cabin     295 non-null    object 
 10  Embarked  1307 non-null   object 
dtypes: float64(3), int64(3), object(5)
memory usage: 155.0+ KB
None
                   Age     Sex  Pclass
PassengerId                           
1            22.000000    male       3
2            38.000000  female       1
3            26.000000  female       3
4            35.000000  female       1
5            35.000000    male       3
6            25.962264    male       3
7    

### 9. Finding couples (3 points).

Based on the code from Lecture 5, build a dataframe of couples. Filter it by survival status: select those couples, in which only one of spouses survived or none of two. Built survival statistics by class, i.e. ratio of the number couples with partial survival or couples which died together, divided by total number of couples in class. If the survival status of one or both of spouses is not known, it must be considered as `0`.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be `Series` with three elements indexed by values from `Pclass` column (see P7 as a reference).

In [70]:
def find_couples(df):
    import re

    """Fill missing age values."""
    # Extract the family names
    family_names = (df
                .replace(re.compile(r'\s+\(.*\)'), '')  # Replace in '' - deleting the first part
                .replace(re.compile("Mrs."), "Mr."))[["Name", "Sex"]] # Getting the husband name by replacing Mrs in Mr
    # Taking only the married women (even if their husbands are not on board)
    family_names = family_names[(family_names.Sex=="female") & family_names.Name.str.contains("Mr.")]

    #  Join the above DF back to original dataframe by defining the husand name as an index
    couples = (df.join(family_names
                        .reset_index()
                        .set_index("Name")["PassengerId"], 
                        on="Name", how="inner")) # Inner join - only the intersection
    # Renaming the women's Passenger_Id column
    couples.rename({"PassengerId":"PassengerId_Spouse"},
               axis=1, inplace=True)
    
    # Getting the women's survival state
    couples = couples.join(df[["Survived"]],
                       on="PassengerId_Spouse", rsuffix="_Spouse")
    
    # Replacing NAs with 0
    couples.fillna({'Survived':0,'Survived_Spouse':0}, inplace=True)

    # Counting couples survival
    couples['JointSurvival'] = couples['Survived'] + couples['Survived_Spouse']
    couples_with_part_survival = couples.loc[couples.JointSurvival < 2, :]

    # Grouping by Pclass
    couple_survival_by_class = couples_with_part_survival.groupby(['Pclass']).size()/couples.groupby('Pclass').size()

    # Replacing the series name to Pclass and removing index name for the output format 
    couple_survival_by_class.rename('Pclass', inplace=True)
    couple_survival_by_class.index.name = None

    return couple_survival_by_class

In [71]:
PROBLEM_ID = 9

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, find_couples)

In [72]:
if not TEST:
    print(find_couples(titanic))

1    0.780488
2    0.956522
3    1.000000
Name: Pclass, dtype: float64


In [73]:
if TEST:
    print(f"{STUDENT}: {int(100 * total_grade / MAX_POINTS)}")